In [45]:
import os
import shutil
import random

def split_dataset_three(
    source_dir,
    output_dir,
    classes,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    seed=42
):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, \
        "Ratios must sum to 1.0"

    random.seed(seed)

    # Output dirs
    train_dir = os.path.join(output_dir, "train")
    val_dir = os.path.join(output_dir, "val")
    test_dir = os.path.join(output_dir, "test")

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for cls in classes:
        src_class_dir = os.path.join(source_dir, cls)

        if not os.path.exists(src_class_dir):
            print(f"⚠️ Class not found: {src_class_dir}")
            continue

        files = [
            f for f in os.listdir(src_class_dir)
            if os.path.isfile(os.path.join(src_class_dir, f))
        ]

        random.shuffle(files)

        n = len(files)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)

        train_files = files[:n_train]
        val_files = files[n_train:n_train+n_val]
        test_files = files[n_train+n_val:]

        # Create subfolders
        dst_train = os.path.join(train_dir, cls)
        dst_val   = os.path.join(val_dir, cls)
        dst_test  = os.path.join(test_dir, cls)

        os.makedirs(dst_train, exist_ok=True)
        os.makedirs(dst_val, exist_ok=True)
        os.makedirs(dst_test, exist_ok=True)

        # Copy files
        for f in train_files:
            shutil.copy(os.path.join(src_class_dir, f), os.path.join(dst_train, f))

        for f in val_files:
            shutil.copy(os.path.join(src_class_dir, f), os.path.join(dst_val, f))

        for f in test_files:
            shutil.copy(os.path.join(src_class_dir, f), os.path.join(dst_test, f))

        print(f"✓ {cls}: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

    print("\nDone! Created:")
    print(f" → {train_dir}")
    print(f" → {val_dir}")
    print(f" → {test_dir}")

YOUR_CLASSES = ['bicycle', 'boat', 'bus', 'car', 'helicopter', 'motorcycle', 'truck']

split_dataset_three(
    source_dir="./blurred_cv2",
    output_dir="./vehicle_dataset2",
    classes=YOUR_CLASSES,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)

✓ bicycle: 145 train, 31 val, 32 test
✓ boat: 188 train, 40 val, 41 test
✓ bus: 198 train, 42 val, 44 test
✓ car: 210 train, 45 val, 45 test
✓ helicopter: 210 train, 45 val, 45 test
✓ motorcycle: 72 train, 15 val, 17 test
✓ truck: 210 train, 45 val, 45 test

Done! Created:
 → ./vehicle_dataset2/train
 → ./vehicle_dataset2/val
 → ./vehicle_dataset2/test


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models

# Path to your pretrained CNN
model_path = "mobilenet2.h5"
model = load_model(model_path)
YOUR_CLASSES = ['bicycle', 'boat', 'bus', 'car', 'helicopter', 'motorcycle', 'truck']

2025-12-11 12:38:18.399651: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-11 12:38:18.399758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-11 12:38:18.402536: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-11 12:38:18.427335: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-11 12:38:24.915588: I tensorflow/core/common_

In [2]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import Model

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
outputs = Dense(7, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=outputs)

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'vehicle_dataset2/train',
    target_size=(160, 160),  # match your model input
    batch_size=4,
    classes=YOUR_CLASSES,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'vehicle_dataset2/val',
    target_size=(160, 160),
    batch_size=4,
    classes=YOUR_CLASSES,
    class_mode='categorical'
)

Found 1233 images belonging to 7 classes.
Found 263 images belonging to 7 classes.


In [4]:
for layer in model.layers[:-2]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [5]:
# train top layers (freeze others)
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)
model.save("fine_tuned_blurry_model2.h5")
print("Fine-tuned model saved.")

Epoch 1/10


2025-12-11 12:38:57.034838: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2025-12-11 12:38:57.833927: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 59.05MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-11 12:38:57.844570: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 59.05MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-12-11 12:38:57.902558: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 73.40MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could

309/309 [==============================] - 91s 264ms/step - loss: 1.0639 - accuracy: 0.6188 - val_loss: 0.5445 - val_accuracy: 0.8251
Epoch 2/10
309/309 [==============================] - 79s 255ms/step - loss: 0.3977 - accuracy: 0.8800 - val_loss: 0.3494 - val_accuracy: 0.8859
Epoch 3/10
309/309 [==============================] - 76s 246ms/step - loss: 0.2910 - accuracy: 0.9067 - val_loss: 0.2744 - val_accuracy: 0.9202
Epoch 4/10
309/309 [==============================] - 79s 256ms/step - loss: 0.2547 - accuracy: 0.9148 - val_loss: 0.2772 - val_accuracy: 0.8859
Epoch 5/10
309/309 [==============================] - 76s 248ms/step - loss: 0.2138 - accuracy: 0.9319 - val_loss: 0.2850 - val_accuracy: 0.8669
Epoch 6/10
309/309 [==============================] - 79s 255ms/step - loss: 0.1694 - accuracy: 0.9489 - val_loss: 0.2725 - val_accuracy: 0.8783
Epoch 7/10
309/309 [==============================] - 76s 248ms/step - loss: 0.1668 - accuracy: 0.9424 - val_loss: 0.2380 - val_accuracy: 0.8

/opt/conda/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Fine-tuned model saved.


In [ ]:
# Path to fine-tuned model
model = load_model("fine_tuned_blurry_model.h5")

# Folder with all test images (no class subfolders needed)
test_folder = "vehicle_dataset/test"
img_size = (160, 160)  # same as model input

test_files = [f for f in os.listdir(test_folder) if f.lower().endswith((".png", ".jpg", ".jpeg"))]

test_images = []
file_names = []

for f in test_files:
    img_path = os.path.join(test_folder, f)
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0  # rescale like training
    test_images.append(img_array)
    file_names.append(f)

test_images = np.array(test_images)

In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.signal import convolve2d
from skimage import restoration
import os
from tensorflow.keras.models import load_model
import pandas as pd
from pathlib import Path

def predict_single_image(model, img_path, class_names):
    img = cv2.imread(img_path)
    if img is None:
        return None, None

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img, verbose=0)
    idx = np.argmax(preds[0])
    confidence = preds[0][idx]

    return class_names[idx], confidence

def run_predictions_finetuned(model, data_dir, class_names):
    print("\nRunning predictions on fine-tuned model...")
    print("="*60)

    results = []
    correct = 0
    total = 0

    for true_class in class_names:
        class_path = os.path.join(data_dir, true_class)

        if not os.path.exists(class_path):
            continue

        images = os.listdir(class_path)
        if not images:
            continue

        print(f"\n  {true_class}/ ({len(images)} images)")
        class_correct = 0

        for i, img_file in enumerate(images):
            if i % 20 == 0 and i > 0:
                print(f"    {i}/{len(images)}...")

            img_path = os.path.join(class_path, img_file)

            pred_class, conf = predict_single_image(model, img_path, class_names)
            if pred_class is None:  
                continue

            is_correct = (pred_class == true_class)
            if is_correct:
                class_correct += 1
                correct += 1

            total += 1
            results.append({
                "true_class": true_class,
                "filename": img_file,
                "predicted_class": pred_class,
                "confidence": conf,
                "correct": is_correct
            })

        print(f"    ✓ {class_correct}/{len(images)} "
              f"= {class_correct/len(images)*100:.1f}%")

    overall = (correct / total) * 100 if total > 0 else 0
    print(f"\nOverall accuracy: {correct}/{total} = {overall:.2f}%")

    return pd.DataFrame(results), overall

In [9]:
model = load_model("fine_tuned_blurry_model2.h5")
df, acc = run_predictions_finetuned(
    model,
    data_dir="vehicle_dataset2/test",
    class_names=YOUR_CLASSES
)


Running predictions on fine-tuned model...

  bicycle/ (32 images)
    20/32...
    ✓ 32/32 = 100.0%

  boat/ (41 images)
    20/41...
    40/41...
    ✓ 40/41 = 97.6%

  bus/ (44 images)
    20/44...
    40/44...
    ✓ 42/44 = 95.5%

  car/ (45 images)
    20/45...
    40/45...
    ✓ 45/45 = 100.0%

  helicopter/ (45 images)
    20/45...
    40/45...
    ✓ 44/45 = 97.8%

  motorcycle/ (17 images)
    ✓ 15/17 = 88.2%

  truck/ (45 images)
    20/45...
    40/45...
    ✓ 42/45 = 93.3%

Overall accuracy: 260/269 = 96.65%


In [10]:
df2, acc2 = run_predictions_finetuned(
    model,
    data_dir="vehicle_dataset2/val",
    class_names=YOUR_CLASSES
)


Running predictions on fine-tuned model...

  bicycle/ (31 images)
    20/31...
    ✓ 31/31 = 100.0%

  boat/ (40 images)
    20/40...
    ✓ 37/40 = 92.5%

  bus/ (42 images)
    20/42...
    40/42...
    ✓ 41/42 = 97.6%

  car/ (45 images)
    20/45...
    40/45...
    ✓ 45/45 = 100.0%

  helicopter/ (45 images)
    20/45...
    40/45...
    ✓ 41/45 = 91.1%

  motorcycle/ (15 images)
    ✓ 14/15 = 93.3%

  truck/ (45 images)
    20/45...
    40/45...
    ✓ 38/45 = 84.4%

Overall accuracy: 247/263 = 93.92%
